In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
import streamlit as st
st.set_page_config(page_title="Pintar - Document Parser")
st.title(f"{os.getenv('TITLE')}")

from datetime import datetime
from dateutil.parser import parse
import json
from PIL import Image
import pandas as pd
import base64
from io import BytesIO
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
    
# from process import get_raw_text_from_pages, get_processed_text_from_pages
from utils import convert_pdf_to_image, get_img, save_csv_file, delete_file

from GCP import upload

2024-08-02 18:02:18.638 
  command:

    streamlit run /home/abdel_taha/projects/document_parser_demo_streamlit_app/dpenv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [2]:
def get_pages(uploaded_file):
    filename = uploaded_file.name.split(".")[0]
    if uploaded_file.name.split(".")[1].lower() == "pdf":
        pdf_pages = convert_pdf_to_image(uploaded_file.read())
    else:
        pdf_pages = [get_img(uploaded_file.read())]
    
    return pdf_pages, filename

In [3]:
def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Re-sized Base64 string
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

In [9]:
from PIL import Image

img = Image.open("Invoice_1.png")
image_b64 = convert_to_base64(img)

In [4]:
llm = ChatOllama(model="llava:13b", temperature=0)

In [5]:
def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]

In [7]:
from langchain_core.output_parsers import StrOutputParser

In [8]:
chain = prompt_func | llm | StrOutputParser()

In [14]:
query_chain = chain.invoke(
    {"text": "How many items in this invoice?", "image": image_b64}
)

print(query_chain)

 The image you've provided appears to be a screenshot of an invoice or financial document. However, due to the resolution and angle of the photo, it is not possible for me to accurately count the number of items on this invoice. If you need assistance with analyzing the content of the invoice, please provide a clearer image or describe the details that you would like me to address.
